In [111]:
import numpy as np
import pandas as pd
import re
import nltk

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer as stemmer

from tqdm.notebook import tqdm

In [112]:
incidents_fn = "data/source/20220413_D1_Incidents.csv"
rule_book_fn = "data/rule_book.csv"

In [113]:
rules = pd.read_csv(rule_book_fn)
incidents = pd.read_csv(incidents_fn, dtype=str)

In [114]:
incidents.head(3)

,tblID,IRN,IncidentNumber,IncidentDate,TimeofIncident,BusinessUnit,BusinessGroup,ServiceLine,Project,Office,...,JFCAction,OHdefinition,MTCcount,RWCcount,LTIcount,FAcount,TransferTimeStamp,SourceFileName,FullDescription,ImmediateAction
0,175,39878,20200472,26/02/2020,14:15,Projects,Renewable Energy & Power,Power & Industrials EPC,YCI M1 Project,YCI M1 Project,...,NaN,NaN,NaN,NaN,NaN,1,NaN,CAIRS_Incidents.csv,Employee was grinding with a four-inch grinder...,Employee was transported to on-site medical an...
1,176,39877,20200471,26/02/2020,16:15,Consulting,Energy Optimisation & Innovation,EMEA,E&II Sustainable Infrastructure,E&II Sustainable Infrastructure,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAIRS_Incidents.csv,See supporting information,"I stopped the drillers from installing, phoned..."
2,177,39876,20200470,27/02/2020,10:15,Operations,Asia Pacific,APAC East,Esso PNG LNG,Hides,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAIRS_Incidents.csv,At approximately 10:15 on the 27th of Feb a hy...,"All contaminated materials, absorbent pads, we..."


In [115]:
incidents.rename(columns={'IncidentNumber': 'incident_id'}, inplace=True)
incidents['text'] = (
        incidents['ShortDescription'].astype(str).fillna('') + ' ' + 
        incidents['FullDescription'].astype(str).fillna('') + ' ' + 
        incidents['ImmediateAction'].astype(str).fillna('')
).str.lower()

incidents = incidents[['incident_id', 'text']]

In [116]:
incidents.text[0]

"foreign body entered employee's (l) eye while grinding. employee was grinding with a four-inch grinder on an iron support in the compressor building when he felt discomfort to his left eye. employee was wearing all required ppe. investigation is pending upon employee’s return from offsite medical evaluation. it will be noted that the site has been dealing with strong winds throughout the day. foreign body was removed using first aid measures. employee was transported to on-site medical and then transferred to an off-site eye specialist for further treatment. foreign body was removed using first aid measures. employee was released to return to work without restrictions."

In [117]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

test = incidents.text.head(1).apply(lemmatize_text)
print(list(test))

[['foreign', 'body', 'entered', "employee's", '(l)', 'eye', 'while', 'grinding.', 'employee', 'wa', 'grinding', 'with', 'a', 'four-inch', 'grinder', 'on', 'an', 'iron', 'support', 'in', 'the', 'compressor', 'building', 'when', 'he', 'felt', 'discomfort', 'to', 'his', 'left', 'eye.', 'employee', 'wa', 'wearing', 'all', 'required', 'ppe.', 'investigation', 'is', 'pending', 'upon', 'employee’s', 'return', 'from', 'offsite', 'medical', 'evaluation.', 'it', 'will', 'be', 'noted', 'that', 'the', 'site', 'ha', 'been', 'dealing', 'with', 'strong', 'wind', 'throughout', 'the', 'day.', 'foreign', 'body', 'wa', 'removed', 'using', 'first', 'aid', 'measures.', 'employee', 'wa', 'transported', 'to', 'on-site', 'medical', 'and', 'then', 'transferred', 'to', 'an', 'off-site', 'eye', 'specialist', 'for', 'further', 'treatment.', 'foreign', 'body', 'wa', 'removed', 'using', 'first', 'aid', 'measures.', 'employee', 'wa', 'released', 'to', 'return', 'to', 'work', 'without', 'restrictions.']]


In [10]:
lemmatizer.lemmatize('employee’s')

'employee’s'

In [11]:
stem = stemmer()
def stem_text(text):
    return [stem.stem(w) for w in w_tokenizer.tokenize(text)]
   
test = incidents.text.head(1).apply(stem_text)
print(list(test))

[['foreign', 'bodi', 'enter', "employee'", '(l)', 'eye', 'while', 'grinding.', 'employe', 'wa', 'grind', 'with', 'a', 'four-inch', 'grinder', 'on', 'an', 'iron', 'support', 'in', 'the', 'compressor', 'build', 'when', 'he', 'felt', 'discomfort', 'to', 'hi', 'left', 'eye.', 'employe', 'wa', 'wear', 'all', 'requir', 'ppe.', 'investig', 'is', 'pend', 'upon', 'employee’', 'return', 'from', 'offsit', 'medic', 'evaluation.', 'it', 'will', 'be', 'note', 'that', 'the', 'site', 'ha', 'been', 'deal', 'with', 'strong', 'wind', 'throughout', 'the', 'day.', 'foreign', 'bodi', 'wa', 'remov', 'use', 'first', 'aid', 'measures.', 'employe', 'wa', 'transport', 'to', 'on-sit', 'medic', 'and', 'then', 'transfer', 'to', 'an', 'off-sit', 'eye', 'specialist', 'for', 'further', 'treatment.', 'foreign', 'bodi', 'wa', 'remov', 'use', 'first', 'aid', 'measures.', 'employe', 'wa', 'releas', 'to', 'return', 'to', 'work', 'without', 'restrictions.']]


In [12]:
def tokenize(text):
    tokens = word_tokenize(text)
    tokens = [tok for tok in tokens if tok!='-']
    return tokens

In [20]:
test_tokens = tokenize(incidents.text[0])
test_tokens

['foreign',
 'body',
 'entered',
 'employee',
 "'s",
 '(',
 'l',
 ')',
 'eye',
 'while',
 'grinding',
 '.',
 'employee',
 'was',
 'grinding',
 'with',
 'a',
 'four-inch',
 'grinder',
 'on',
 'an',
 'iron',
 'support',
 'in',
 'the',
 'compressor',
 'building',
 'when',
 'he',
 'felt',
 'discomfort',
 'to',
 'his',
 'left',
 'eye',
 '.',
 'employee',
 'was',
 'wearing',
 'all',
 'required',
 'ppe',
 '.',
 'investigation',
 'is',
 'pending',
 'upon',
 'employee',
 '’',
 's',
 'return',
 'from',
 'offsite',
 'medical',
 'evaluation',
 '.',
 'it',
 'will',
 'be',
 'noted',
 'that',
 'the',
 'site',
 'has',
 'been',
 'dealing',
 'with',
 'strong',
 'winds',
 'throughout',
 'the',
 'day',
 '.',
 'foreign',
 'body',
 'was',
 'removed',
 'using',
 'first',
 'aid',
 'measures',
 '.',
 'employee',
 'was',
 'transported',
 'to',
 'on-site',
 'medical',
 'and',
 'then',
 'transferred',
 'to',
 'an',
 'off-site',
 'eye',
 'specialist',
 'for',
 'further',
 'treatment',
 '.',
 'foreign',
 'body',
 '

In [13]:
def translate_to_regex(rule_part):
    """
    converts rule_book rules to regex format;
    drops trailing separators in process
    """
    if isinstance(rule_part, str):
        rule_part = re.sub(r"\s{2,}", " ", rule_part)
        rule_part = re.sub(r'\\\\b', r'\\b', rule_part)
        
        return r'(\b' + r'\b)|(\b'.join([s for s in rule_part.split('_ ') if s]) + r'\b)'
    else:
        return ''

In [14]:
rules['keyword'] = rules['keyword'].apply(lambda x: x.replace('*', '.*')+r'\b')
rules['rules_pre'] = rules['rules_pre'].apply(translate_to_regex)
rules['rules_post'] = rules['rules_post'].apply(translate_to_regex)
rules['rules_all'] = rules['rules_all'].apply(translate_to_regex)

rules['voids'] = rules['voids'].apply(translate_to_regex)
print('rules translated')

rules translated


In [15]:
rules

,group,keyword,rules_pre,rules_post,rules_all,voids
0,eye injury or irritation,eye.*\b,(\bsomething.*in\b)|(\bswell.*\b)|(\bswollen\b...,,(\birritation\b)|(\bforeign body\b)|(\bforeign...,
1,site compliance or practice issue,permit\b,(\bwithout.*work\b)|(\bnot captured\b),(\bnot in place\b)|(\bwere not captured\b),(\bduring.* audit\b)|(\bfailed.* section\b)|(\...,


In [16]:
def check_presence(pattern, string):
    if pattern:
        return bool(re.search(pattern, string))
    else:
        return False

In [17]:
def find_pattern(tokens, keyword, check_pre, check_post, check_all, check_void, window):
    """
    for a list of tokens finds specified keyword and returns True
    if the neighbourhood of this keyword satisfies pre-, post- or all- context rules
    and doesn't contain anything forbidden
    :param tokens: list of tokens
    :param keyword: pattern which a token should match
    :param check_pre: pattern which several previous tokens (concatenated) should match
    :param check_post: pattern which several subsequent tokens (concatenated) should match
    :param check_all: pattern which previous tokens + keyword + subsequent tokens should match
    :param check_void: pattern which previous tokens + keyword + subsequent tokens should NOT match
    :param window: N of pre and post tokens to consider
    :return: True/False - whether at least one matching part was found
    """
    # Extract contexts of keyword (if any found)
    # Four parts: [(pre), (keyword), (post), (all)]
    matches = [(
        ' '.join(tokens[max(0, i - window - 1): i]), tokens[i],
        ' '.join(tokens[i + 1:i + window + 1]),
        ' '.join(tokens[max(0, i - window - 1):i + window + 1])
        ) for i, tok in enumerate(tokens) if re.match(keyword, tok)]
    
    # do tests
    final_match = any([
        (check_presence(check_pre, pre) or check_presence(check_all, all_) or check_presence(check_post, post))
        and not check_presence(check_void, all_) for (pre, kw, post, all_) in matches
    ])
    return final_match

In [21]:
tokens = test_tokens
keyword = rules.keyword[0]
check_pre = rules.rules_pre[0]
check_post = rules.rules_post[0]
check_all = rules.rules_all[0]
check_void = rules.voids[0]
window = 5

In [22]:
matches = [(
        ' '.join(tokens[max(0, i - window - 1): i]), # pre-context
            tokens[i],                               # keyword
        ' '.join(tokens[i + 1:i + window + 1]),      # post-context
        ' '.join(tokens[max(0, i - window - 1):i + window + 1]) # All context
        ) for i, tok in enumerate(tokens) if re.match(keyword, tok)]

In [23]:
matches

[("entered employee 's ( l )",
  'eye',
  'while grinding . employee was',
  "entered employee 's ( l ) eye while grinding . employee was"),
 ('he felt discomfort to his left',
  'eye',
  '. employee was wearing all',
  'he felt discomfort to his left eye . employee was wearing all'),
 ('and then transferred to an off-site',
  'eye',
  'specialist for further treatment .',
  'and then transferred to an off-site eye specialist for further treatment .')]

In [24]:
find_pattern(
    tokens = test_tokens, 
    keyword = rules.keyword[0], 
    check_pre = rules.rules_pre[0], 
    check_post = rules.rules_post[0], 
    check_all = rules.rules_all[0], 
    check_void = rules.voids[0], 
    window = 5
)

True

In [25]:
[(check_presence(check_pre, pre) or check_presence(check_all, all_) or check_presence(check_post, post))
        and not check_presence(check_void, all_) for (pre, kw, post, all_) in matches]

[False, True, False]

In [26]:
for (pre, kw, post, all_) in matches:
    #print(pre)
    #print(kw)
    #print(post)
    print(all_)

entered employee 's ( l ) eye while grinding . employee was
he felt discomfort to his left eye . employee was wearing all
and then transferred to an off-site eye specialist for further treatment .


In [27]:
check_all

'(\\birritation\\b)|(\\bforeign body\\b)|(\\bforeign object\\b)|(\\birritation (.*) eye\\b)|(\\bpain in (.*) eye\\b)|(\\bfell (.*) eye\\b)|(\\bfall (.*) eye\\b)|(\\bdebris (.*) eye\\b)|(\\bpain to\\b)'

In [28]:
# Sentence tokenizer
print(sent_tokenize(incidents.text[0]))

["foreign body entered employee's (l) eye while grinding.", 'employee was grinding with a four-inch grinder on an iron support in the compressor building when he felt discomfort to his left eye.', 'employee was wearing all required ppe.', 'investigation is pending upon employee’s return from offsite medical evaluation.', 'it will be noted that the site has been dealing with strong winds throughout the day.', 'foreign body was removed using first aid measures.', 'employee was transported to on-site medical and then transferred to an off-site eye specialist for further treatment.', 'foreign body was removed using first aid measures.', 'employee was released to return to work without restrictions.']


In [81]:
syn_csv = pd.read_csv('synonyms.csv')
syn_csv

,syn,keywords
0,worker_syns,"employee, ip, worker, man, woman, he, she, his..."
1,pain_syns,"pain, discomfort, soreness, hurt, something in..."
2,eye_syns,"eye, pupil, iris"


In [68]:
re.sub(r", ", "', '", syn_csv.keywords[0])

"employee', 'ip', 'worker', 'man', 'woman', 'he', 'she', 'his', 'her', 'the', 'left', 'right"

In [105]:
# Load the dictionary of synonyms
syn_dict = {}
for r in range(len(syn_csv)):
    syn_toks = tokenize(re.sub(r", ", " ", syn_csv.keywords[r]))
    syn_dict.update({syn_csv.syn[r]:syn_toks})

In [106]:
syn_dict

{'worker_syns': ['employee',
  'ip',
  'worker',
  'man',
  'woman',
  'he',
  'she',
  'his',
  'her',
  'the',
  'left',
  'right'],
 'pain_syns': ['pain',
  'discomfort',
  'soreness',
  'hurt',
  'something',
  'in',
  'grit',
  'swelling',
  'swollen',
  'dust',
  'particle',
  'irritation',
  'foreign',
  'fb',
  'debris'],
 'eye_syns': ['eye', 'pupil', 'iris']}

In [30]:
chk_text = 'employee was grinding with a four-inch grinder on an iron support in the compressor building when he felt discomfort to his left eye.'
pattern = '(\\bemployee (.*) discomfort (.*) eye\\b)'
check_presence(pattern, chk_text)

True

In [31]:
sents = sent_tokenize(incidents.text[0])
for sent in sents:
    if check_presence(pattern, sent):
        print(f'{check_presence(pattern, sent)}: {sent}')

True: employee was grinding with a four-inch grinder on an iron support in the compressor building when he felt discomfort to his left eye.


In [32]:
chk_text = 'the ip felt extreme soreness in his eye'
finds_list = []
finds_pats = []
incid_nums = []
for worker_syn in syn_dict.get('worker_syns'):
    for pain_syn in syn_dict.get('pain_syns'): 
        for eye_syn in syn_dict.get('eye_syns'):   
            pattern = f'(\\b{worker_syn} (.*) {pain_syn} (.*) {eye_syn}\\b)'
            check = check_presence(pattern, chk_text)
            if check: 
                print(f'{check}: {pattern}')
                finds_list.append(check)
                finds_pats.append(pattern)
                #incid_nums.append(irn)

True: (\bip (.*) soreness (.*) eye\b)


In [100]:
pos_1st = syn_dict.get('worker_syns')
pos_2nd = syn_dict.get('pain_syns')
pos_3rd = syn_dict.get('eye_syns')

# Connections between synonyms 1-2, 2-3
connect = ['(.*)', '(.*)']

# Shuffle rules
srs = ([2, 1, 4, 3, 0], [])

finds_list = []
finds_pats = []
incid_nums = []

for row in tqdm(range(len(incidents))):
    par_text = incidents.text[row].lower()
    # Check sentence by sentence, don't use span
    # TODO: Check impact of using span, should it be used or not?
    sen_toks = sent_tokenize(par_text)    
    irn = incidents.incident_id[row]
    for chk_text in sen_toks:
        # TODO: remove plurals, workers or worker's >>> worker
        for first_syn in pos_1st:
            locals()["first_syn"] = first_syn
            for second_syn in pos_2nd: 
                for third_syn in pos_3rd:  
                    x = f'{first_syn} {connect[0]} {second_syn} {connect[1]} {third_syn}'
                    pattern = f'(\\b{x}\\b)'
                    check = check_presence(pattern, chk_text)
                    if check: 
                        print(f'{check}: {pattern}')
                        finds_list.append(check)
                        finds_pats.append(pattern)
                        incid_nums.append(irn)
                    else:
                        for sr in srs:
                            if len(sr) == 0: continue
                            a = x.split()
                            # Adjust the word sequence using shuffle rule
                            pattern = f'(\\b{a[sr[0]]} {a[sr[1]]} {a[sr[2]]} {a[sr[3]]} {a[sr[4]]}\\b)'
                            rev_check = check_presence(pattern, chk_text)
                            if rev_check: 
                                print(f'{rev_check}: {pattern}')
                                finds_list.append(check)
                                finds_pats.append(pattern)
                                incid_nums.append(irn)
                    

  0%|          | 0/27158 [00:00<?, ?it/s]

True: (\bemployee (.*) discomfort (.*) eye\b)
True: (\bemployee (.*) in (.*) eye\b)
True: (\bhe (.*) discomfort (.*) eye\b)
True: (\bthe (.*) discomfort (.*) eye\b)
True: (\bin (.*) eye (.*) he\b)
True: (\bhe (.*) foreign (.*) eye\b)
True: (\bhis (.*) in (.*) eye\b)
True: (\bhis (.*) foreign (.*) eye\b)
True: (\bthe (.*) in (.*) eye\b)
True: (\bthe (.*) foreign (.*) eye\b)
True: (\bleft (.*) in (.*) eye\b)
True: (\bleft (.*) foreign (.*) eye\b)
True: (\bright (.*) in (.*) eye\b)
True: (\bright (.*) foreign (.*) eye\b)
True: (\bthe (.*) debris (.*) eye\b)
True: (\bemployee (.*) discomfort (.*) eye\b)
True: (\bhis (.*) discomfort (.*) eye\b)
True: (\bthe (.*) discomfort (.*) eye\b)
True: (\bip (.*) irritation (.*) eye\b)
True: (\bemployee (.*) something (.*) eye\b)
True: (\bhe (.*) something (.*) eye\b)
True: (\bhis (.*) something (.*) eye\b)
True: (\bforeign (.*) eye (.*) the\b)
True: (\bemployee (.*) discomfort (.*) eye\b)
True: (\bip (.*) discomfort (.*) eye\b)
True: (\bip (.*) in (.*

True: (\bdiscomfort (.*) eye (.*) his\b)
True: (\bhis (.*) irritation (.*) eye\b)
True: (\bip (.*) in (.*) eye\b)
True: (\bip (.*) irritation (.*) eye\b)
True: (\birritation (.*) eye (.*) his\b)
True: (\bthe (.*) in (.*) eye\b)
True: (\bthe (.*) irritation (.*) eye\b)
True: (\birritation (.*) eye (.*) left\b)
True: (\bip (.*) foreign (.*) eye\b)
True: (\bthe (.*) foreign (.*) eye\b)
True: (\bemployee (.*) irritation (.*) eye\b)
True: (\birritation (.*) eye (.*) his\b)
True: (\bemployee (.*) in (.*) eye\b)
True: (\bhe (.*) debris (.*) eye\b)
True: (\bthe (.*) debris (.*) eye\b)
True: (\bin (.*) eye (.*) he\b)
True: (\bemployee (.*) debris (.*) eye\b)
True: (\bdebris (.*) eye (.*) the\b)
True: (\bthe (.*) something (.*) eye\b)
True: (\bthe (.*) in (.*) eye\b)
True: (\bin (.*) eye (.*) ip\b)
True: (\bforeign (.*) eye (.*) ip\b)
True: (\bthe (.*) in (.*) eye\b)
True: (\bthe (.*) foreign (.*) eye\b)
True: (\bforeign (.*) eye (.*) employee\b)
True: (\bip (.*) foreign (.*) eye\b)
True: (\bthe

True: (\bemployee (.*) in (.*) eye\b)
True: (\bhe (.*) discomfort (.*) eye\b)
True: (\bhe (.*) in (.*) eye\b)
True: (\bdiscomfort (.*) eye (.*) his\b)
True: (\bhis (.*) in (.*) eye\b)
True: (\bdiscomfort (.*) eye (.*) the\b)
True: (\bin (.*) eye (.*) the\b)
True: (\bleft (.*) in (.*) eye\b)
True: (\bemployee (.*) in (.*) eye\b)
True: (\bshe (.*) in (.*) eye\b)
True: (\bher (.*) in (.*) eye\b)
True: (\bthe (.*) in (.*) eye\b)
True: (\bworker (.*) dust (.*) eye\b)
True: (\bworker (.*) something (.*) eye\b)
True: (\bworker (.*) in (.*) eye\b)
True: (\bsomething (.*) eye (.*) his\b)
True: (\bin (.*) eye (.*) his\b)
True: (\bhe (.*) in (.*) eye\b)
True: (\bemployee (.*) in (.*) iris\b)
True: (\bip (.*) in (.*) eye\b)
True: (\bip (.*) dust (.*) eye\b)
True: (\bip (.*) irritation (.*) eye\b)
True: (\bhe (.*) in (.*) eye\b)
True: (\bdust (.*) eye (.*) he\b)
True: (\bhe (.*) irritation (.*) eye\b)
True: (\bhis (.*) in (.*) eye\b)
True: (\bdust (.*) eye (.*) his\b)
True: (\bhis (.*) irritation (

KeyboardInterrupt: 

In [101]:
len(list(dict.fromkeys(incid_nums)))

93

In [102]:
# Inspect findings
treat = False
for inc in list(dict.fromkeys(incid_nums)):
    par_text = incidents.text[incidents.incident_id == inc].values[0]    
    sen_toks = sent_tokenize(par_text) 
    print(inc)
    for sen in sen_toks: 
        if treat:
            tokenized_words = word_tokenize(sen)
            tokenized_sentence = []
            for word in tokenized_words:
                tokenized_sentence.append(lemmatizer.lemmatize(word))
                #tokenized_sentence.append(stem.stem(word))
            tokenized_sentence = " ".join(tokenized_sentence)                
            print(tokenized_sentence)
        else:
            print(sen)    
    print('\n')    

20200472
foreign body entered employee's (l) eye while grinding.
employee was grinding with a four-inch grinder on an iron support in the compressor building when he felt discomfort to his left eye.
employee was wearing all required ppe.
investigation is pending upon employee’s return from offsite medical evaluation.
it will be noted that the site has been dealing with strong winds throughout the day.
foreign body was removed using first aid measures.
employee was transported to on-site medical and then transferred to an off-site eye specialist for further treatment.
foreign body was removed using first aid measures.
employee was released to return to work without restrictions.


20200446
individual reported to medic with blurred vision in right eye during the tea break on the afternoon of friday 21st feb 2020 at 15:00 individual was using their mobile phone and noticed their vision was blurred in the right eye.the individual already has an existing medical condition within his left ey

In [93]:
stem.stem('Be particles in hands')

'be particles in hand'

In [110]:
org_string = "This is employee's hand"
pattern = r"'s"

# Replace all occurrences of character 's with an empty string
org_string = re.sub(pattern, '', org_string )
print(org_string)

This is employee hand
